In [32]:
import torch
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from dgl.nn import GraphConv,SAGEConv
import dgl
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
import seaborn as sns
import random
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split
import os
import yaml
import time
import datetime
from modules import ICPKGIDataset
from models import PatchGCN

In [3]:
'''
データセットを読み込む
データローダーをシャッフルで作成
モデルにデータを入力
1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る
出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする
テストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う
学習における分類精度と特徴グラフによる分類精度を計算し保存する
分類学習における予測と正解を用いて損失を計算しネットワークを更新する
すべてのテストデータに対して行ったのち全epochについて行う
このファイルは1種類の物体について分類を行うファイルとする
'''

'\nデータセットを読み込む\nデータローダーをシャッフルで作成\nモデルにデータを入力\n1epoch終了後各物体の視線方向ごとのデータを入力し中間層の出力を得る\n出力の特徴マップを同じ位置のノードごとの平均をとってその方向の特徴グラフとする\nテストデータを同様の手順で処理し特徴グラフを取得する。ただしそれぞれのデータで行う\n学習における分類精度と特徴グラフによる分類精度を計算し保存する\n分類学習における予測と正解を用いて損失を計算しネットワークを更新する\nすべてのテストデータに対して行ったのち全epochについて行う\nこのファイルは1種類の物体について分類を行うファイルとする\n'

In [29]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
object_name = 'airplane'  #car bus airplane
setting_file = "config2.yaml"

#データ読み込み
dataset=ICPKGIDataset(f'../data/ICPKGI/8patch_gray_{object_name}.dgl')

#各クラスから均等に10個ずつテスト用として抜き出しtrainデータセットとtestデータセットを作成
labels=[i.item() for _,i in dataset]
traindataset, testdataset, trainlabels, testlabels=train_test_split(dataset,labels,test_size=0.2,shuffle=True,stratify=labels)

#データローダー作成
traindataloader=GraphDataLoader(traindataset,batch_size=16,shuffle=True,num_workers = 0,pin_memory = True)
testdataloader=GraphDataLoader(testdataset,batch_size=10,shuffle=True,num_workers = 0,pin_memory = True)


In [31]:
for i,(g,d) in enumerate(traindataloader):
    break
print(g,d)

Graph(num_nodes=1024, num_edges=6720,
      ndata_schemes={'f': Scheme(shape=(1, 32, 32), dtype=torch.float32)}
      edata_schemes={}) tensor([0, 0, 0, 3, 3, 2, 3, 4, 3, 0, 0, 4, 1, 1, 1, 1])


In [43]:
#設定ファイル読み込み
with open(f'./configs/{setting_file}','r') as f:
    config = yaml.safe_load(f)

#モデルの初期化
model=PatchGCN(1024,[8],5,embedding=True)
model.to(device)

PatchGCN(
  (input_layer): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_neigh): Linear(in_features=1024, out_features=8, bias=False)
    (fc_self): Linear(in_features=1024, out_features=8, bias=True)
  )
  (middle_layers): ModuleList()
  (output_layer): GraphConv(in=8, out=5, normalization=both, activation=None)
  (m): LeakyReLU(negative_slope=0.01)
  (flatt): Flatten(start_dim=1, end_dim=-1)
)

In [44]:
for i,(batched_graph,labels) in enumerate(traindataloader):
    batched_graph = batched_graph.to(device)
    labels = labels.to(device)

    pred,emb = model(batched_graph,batched_graph.ndata['f'])

    break
print(i)

0


In [46]:
print(emb.shape)

torch.Size([16, 8])
